In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from voxel_grid_dataset.dataset import TwoVoxelGridsTwoTransformsDataset
from voxel_grid_neural_compression.neural_netrworks import VoxelGridCompressor
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

dataset = TwoVoxelGridsTwoTransformsDataset()
dataset.load_dataset()
dataloader = torch.utils.data.DataLoader(dataset)

Loading Dataset


100%|██████████| 10000/10000 [08:10<00:00, 20.37it/s]


In [4]:
criterion = nn.MSELoss(6)

# Training loop
trainloader = DataLoader(dataset, batch_size=16, shuffle=True)
writer = SummaryWriter("/home/lorenzo/.tensorboard")
counter = 0
for lr in [0.01,0.005,0.001,0.0005,0.0001,0.00005]:
    model = VoxelGridCompressor().to("cuda")
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    for epoch in tqdm(range(32)):  # Change the number of epochs as needed
        for vg1, vg2, labels in tqdm(trainloader,leave=False):
            optimizer.zero_grad()
            vg1 = vg1.to("cuda")
            vg2 = vg2.to("cuda")
            labels = labels.to("cuda")
            outputs = model(vg1, vg2)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            writer.add_scalar(f"loss_per_iter_lr{lr}", loss, counter)
            counter += 1
    torch.save(model.state_dict(), f"/home/lorenzo/models/a{lr}.torch")

/usr/lib/python3/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))
  0%|          | 0/32 [00:00<?, ?it/s]

2


2


2


2


...

2


2


  0%|          | 0/32 [00:03<?, ?it/s]


KeyboardInterrupt: 

: 